# Cellular Automata Logbook 
### Student number: 20077188

###### Code included in this program is written so as to display the automata frame by frame rather than as animations, as this will be the code implemented in the formal report.

### 28/01/2022

I have decided to begin the project by dedicating some time to familiarising myself with the concept of Cellular Automata. I especially read up on John Horton Conway’s game of life and explored myriad online ressources that demonstrated the simulation. 

As a preliminary idea, I explored the various methods of constructions grids in matplotlib as this would appear as the logical host for the structural nature of the project. Though, the function `plt.imshow` seems to provide the necessary tools to implement all necessary information.

### 29/01/2022

For the first part of the project, I have decided to play around with some ideas to help figure out the most effective way to approach the rendering of the simple singular space and time dimension automata.

The logical way to proceed seems to be the construction a single row, implemented as an `np.array` function. This corresponds to the singular time dimension. Hence we will initialise a certain amount of live and dead cells, corresponding to values of 1 and 0 respectively in the row.

As time evolves one can update the row and insert it in a matrix of the form: <b>Spatial size</b> $\times$ <b>Time duration</b>.

### 01/02/2022

I believe that the implementation of two specific functions is required for the program. One will evaluate the rules based on a given total (which can be modified to offer alternate results) and another will apply these rules to every cell in the array.

By applying these two functions together I was able to complete the first part of program and achieve the results displayed in the project booklet.

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Initialising row
space = np.zeros(100)
space[50] = 1 # Placing a single live cell at centre

# Duration of simulation
timesteps = 100

#Initialising matrix view (single dimensions as time evolves)
matrixview = np.zeros((len(space),timesteps))
matrixview[0] = space # Updating first dimension with initialised conditions

def totalrule(total,cellin):
    
    cellout = 0
    
    if total == 0:
        if cellin == 0:
            cellout = 0
        else:
            cellout = 1
        
    if total == 2:
        if cellin == 0:
            cellout = 0
        else:
            cellout = 0
        
    if total == 3:
        if cellin == 0:
            cellout = 1
        else:
            cellout = 1
        
    if total == 1:
        if cellin == 0:
            cellout = 1
        else:
            cellout = 0
            
    return cellout

def update(cv):
    cvu = np.zeros(len(cv))
    for i in range (1, len(cv)-1):
        total = cv[i-1] + cv[i] + cv[i+1]
        cvu[i] = totalrule(total,cv[i])
        
    return cvu

for i in range(1,timesteps):
    u = update(space)
    matrixview[i] = u
    space = u

windowsize = 10
gridfigure = plt.figure(figsize = (windowsize,windowsize))
gridax = gridfigure.subplots()
plt.axis('off')

gridax.imshow(matrixview, aspect='auto', interpolation='nearest', cmap='Greys')

### 03/02/2022

Having discussed my program in the thursday meet up, I realise that my approach to the rule cosntruction was misunderstood. In fact I needed to implement the rules in an entirely different way.

I have now rewritten the code differently. The first will evaluate the rules upon the insertion of a given cell and the sum of itself and two surrounding neighbours. The possible combinations for a 2-state auromato with only the totals of the three cells correspond to:

<br>
<b>
(0,0,0) → total = 0 <br>
(1,0,0) → total = 1 <br>
(0,1,0) → total = 1 <br> 
(0,0,1) → total = 1 <br>
(0,1,1) → total = 2 <br>
(1,0,1) → total = 2 <br>
(1,1,0) → total = 2 <br>
(1,1,1) → total = 3 <br>
</b>
<br>
These totals can then be mapped to certain combinations of rules, in the form:<br>

    def rule(total,cell,rule_value):

        if total == 0:
            cell = rule_value[0]

        elif total == 1:
            cell = rule_value[1]

        elif total == 2: 
            cell = rule_value[2]
            
        elif total == 3: 
            cell = rule_value[3]
            
        return cell
   

Which one could perform on a set of rules of a form such as: <b> {0,1,0,0} </b>.

The second function then updates and returns the row at a following timestep using the implementation of the update function above. This is given by:

Here is the corresponding second function:

    def update(space,rule_value,step):

        space_updated = np.zeros(spacesize)

        for i in range (step, spacesize-step):
            total = space[i-step] + space[i] + space[i+step]
            space_updated[i] = rule(total,space[i],rule_value) 

        return space_updated
        
Finally, I have implemented a `for` loop to evaluate the rules at each timestep: rows are inserted into the display matrix at each step.

It is noted that I have generalised the rules as an array so as to facilitate the cycling between various rules. We would hence need to permute all possible combinations of <b>[0,1]</b> between 4 spaces in order to determine all 16 rules.

Furthermore, there is an additional step variable. This is implemented as a means of experiment, I am curious to see how the rules would change if one were to increase the distance of neighbours on which one performs the rules.

### 06/02/2022

Having explored various rule combinations and fixing boundary conditions, it is now a matter of exploring the system and implementing new extensions. The first modification is to generalise and plot all 16 cobinations. This is done with the cycling of an array containing all possible permutations of <b>[0,1]</b> between 4 spaces.

To add this ability, I have used the external python module `itertools`, that takes in an array and evaluates all possible permutations of the components. I have equally coded my own function to do the same thing, though for the rest of the code I will use the module to facilitate code comprehension. The code below is my own binary digit permuter, it will cycle through all possible combinations of ones and zeros in an array of a given length.

In [ ]:
def permute(length):
    
    values = np.array([0,1])
    
    size = len(values)**length+1
    permutations = np.zeros((size,length))
    
    count = len(values)
    
    spec = 0
    timer = 1
    for j in range(length):
        for i in range(1,size):
            if spec == count:
                spec = 0
            permutations[i-1,length-j-1] = values[spec]
            if timer != 1:
                if i%(timer) == 0: 
                    spec += 1
            else: spec += 1
        timer += timer

    return permutations[:-1]

In [ ]:
permute(4)

I have hence constructed two seperate functions that either plot for a specific rule, or plot all 16 rules subject to initial conditions. 

    • plot_automata(timesteps,spacesize,initial,step,rule)
    
    • plot_automata_16(timesteps,spacesize,initial,step)
    
For the plotting of 16 simultaneous automata, one can now initialise the states with a few lines of code as such:

    timesteps = 100
    spacesize = 100
    initial = np.array([50])
    step = 1
    
Where the initial function correspons to the cell index in the initial space that will be alive.

Whereas for the plotting of a specific rule, one initialises with the addition of the specification of that rule:

    timesteps = 100
    spacesize = 100
    initial = np.array([50])
    step = 1
    rule = (0,1,0,1)
    
A further generalisation is made with regards to the rules where we now take into account the value of the central cell and contribute it to the overall total. This produces a combination of 32 possible rules.

The functions for these are defined exactly analagously with the only key modification in the rule function:

    def rule(total,cell,rule_value):

        total = total + cell
        
        if total == 0:
            cell = rule_value[0]

        elif total == 1:
            cell = rule_value[1]

        elif total == 2: 
            cell = rule_value[2]
            
        elif total == 3: 
            cell = rule_value[3]
            
        elif total == 4:
            cell = rule_value[4]
            
        return cell

# Part 1 - 2-State 1-Dimension 16-Rules

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
from ipywidgets import interactive
import numpy as np
%matplotlib inline

# All rule permutations
import itertools
rules = np.array(list(itertools.product([0, 1], repeat=4)))

def run(rule_value,duration,size,initial_space,step):
    """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 
        for a given set of rules (rule_value)

    Inputs:
    duration       - Duration of simulation
    size           - Size of simulation
    rule_value     - Rule combination
    initial_space  - Initial configuration of live cells
    step           - Neighbour distance to evaluate

    Output:
    matrixview     - Outputs matrix with simulation results

    """   
    
    # Duration of simulation
    timesteps = duration + 1
    spacesize = size + 1

    # Initialising row
    space = np.zeros(spacesize)
    for i in range (len(initial_space)):
        space[initial_space[i]] = 1

    #Initialising matrix view (single dimensions as time evolves)
    matrixview = np.zeros((timesteps,spacesize))
    matrixview[0] = space # Updating first dimension with initialised conditions

    def rule(total,cell,rule_value):
        """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 
            for a given set of rules (rule_value)

        Inputs:
        total          - Sum of neighbouring cells and input cell values
        cell           - Value of input cell
        rule_value     - Rule combination

        Output:
        cell           - Value of the cell in following timestep

        """
        
        if total == 0:
            cell = rule_value[0]

        elif total == 1:
            cell = rule_value[1]

        elif total == 2: 
            cell = rule_value[2]
            
        elif total == 3: 
            cell = rule_value[3]
            
        return cell

    def update(space,rule_value,step):
        """ Evaluates the rules on each cell and its neighbours for an input row (space) 
            and set of rules (rule_value)

        Input:
        space          - Input row 
        rule_value     - Rule combination
        step           - Neighbour distance to evaluate

        Output:
        space_updated  - Output row for following timestep

        """

        # Initialising following row
        space_updated = np.zeros(spacesize)

        # Scanning each cell and its neighbours from inserted row
        for i in range (step, spacesize-step):
            total = space[i-step] + space[i] + space[i+step]
            space_updated[i] = rule(total,space[i],rule_value) # Implementing rule onto the following row

        return space_updated

    # Evaluating the rules for duration of timesteps
    for i in range(1,timesteps):
        space_next = update(space,rule_value,step)
        matrixview[i] = space_next
        space = space_next
        
    return matrixview

def plot_automata_16(timesteps,spacesize,initial,step):
    """ Evaluates the automata for all combinations of rules

    Inputs:
    timesteps          - Duration of simulation
    spacesize          - Size of simulation
    initial            - Initial configuration of live cells
    step               - Step number

    Output:
    Plots automata

    """
    
    # Plotting conditions
    windowsize = 20
    rows = 4
    cols = 4
    gridfigure = plt.figure(figsize = (windowsize,windowsize))
    plt.axis('off')

    # Evaluating for all rules
    for i in range(0,len(rules)):
        matrixout = run(rules[i],timesteps,spacesize,initial,step)
        gridax = gridfigure.add_subplot(rows,cols,i+1)
        gridax.imshow(matrixout, aspect='auto', interpolation='nearest', cmap='Greys')
        gridax.set_title(str(rules[i]))
        plt.axis('off')
        
    plt.show()
    
def plot_automata(timesteps,spacesize,initial,step,rule):
    """ Evaluates the automata for a specific rule

    Inputs:
    timesteps          - Duration of simulation
    spacesize          - Size of simulation
    initial            - Initial configuration of live cells
    step               - Step number
    rule               - Selected rule

    Output:
    Plots automata

    """
    
    # Plotting conditions
    windowsize = 10
    gridfigure = plt.figure(figsize = (windowsize,windowsize))
    plt.axis('off')

    # Evaluating with selected rule
    matrixout = run(rule,timesteps,spacesize,initial,step)
    plt.imshow(matrixout, aspect='auto', interpolation='nearest', cmap='Greys')
    plt.title(str(rule))
    plt.axis('off')
        
    plt.show()
    
# Initial conditions
timesteps = 100
spacesize = 100
initial = np.array([50]) # Placing live cells
step = 1
rule = (0,1,0,1)

### The following functions would then be run

In [ ]:
plot_automata(timesteps,spacesize,initial,step,rule)

In [ ]:
plot_automata_16(timesteps,spacesize,initial,step)

# Part 1 - 2-State 1-Dimension 32-Rules

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
from ipywidgets import interactive
import numpy as np
%matplotlib inline

# 
rules = np.array(list(itertools.product([0, 1], repeat=5)))

def run(rule_value,duration,size,initial_space,step):
    """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 
            for a given set of rules (rule_value)

    Inputs:
    duration       - Duration of simulation
    size           - Size of simulation
    rule_value     - Rule combination
    initial_space  - Initial configuration of live cells
    step           - Neighbour distance to evaluate

    Output:
    matrixview     - Outputs matrix with simulation results

    """   
    # Duration of simulation
    timesteps = duration + 1
    spacesize = size + 1

    # Initialising row
    space = np.zeros(spacesize)
    for i in range (len(initial_space)):
        space[initial_space[i]] = 1

    #Initialising matrix view (single dimensions as time evolves)
    matrixview = np.zeros((timesteps,spacesize))
    matrixview[0] = space # Updating first dimension with initialised conditions

    def rule(total,cell,rule_value):
        """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 
            for a given set of rules (rule_value)

        Inputs:
        total          - Sum of neighbouring cells and input cell values
        cell           - Value of input cell
        rule_value     - Rule combination

        Output:
        cell           - Value of the cell in following timestep

        """   
        
        total = total + cell
        
        if total == 0:
            cell = rule_value[0]

        elif total == 1:
            cell = rule_value[1]

        elif total == 2: 
            cell = rule_value[2]
            
        elif total == 3: 
            cell = rule_value[3]
            
        elif total == 4:
            cell = rule_value[4]
            
        return cell

    def update(space,rule_value,step):
        """ Evaluates the rules on each cell and its neighbours for an input row (space) 
            and set of rules (rule_value)

        Input:
        space          - Input row 
        rule_value     - Rule combination
        step           - Neighbour distance to evaluate

        Output:
        space_updated  - Output row for following timestep

        """

        # Initialising following row
        space_updated = np.zeros(spacesize)

        # Scanning each cell and its neighbours from inserted row
        for i in range (step, spacesize-step):
            total = space[i-step] + space[i] + space[i+step]
            space_updated[i] = rule(total,space[i],rule_value) # Implementing rule onto the following row

        return space_updated

    # Evaluating the rules for duration of timesteps
    for i in range(1,timesteps):
        space_next = update(space,rule_value,step)
        matrixview[i] = space_next
        space = space_next
        
    return matrixview

def plot_automata_32(timesteps,spacesize,initial,step):
    """ Evaluates the automata for all combinations of rules

    Inputs:
    timesteps          - Duration of simulation
    spacesize          - Size of simulation
    initial            - Initial configuration of live cells
    step               - Step number

    Output:
    Plots automata

    """
    
    # Plotting conditions
    windowsize = 20
    rows = 8
    cols = 4
    gridfigure = plt.figure(figsize = (windowsize,windowsize*2))
    plt.axis('off')

    # Evaluating for all rules
    for i in range(0,len(rules)):
        matrixout = run(rules[i],timesteps,spacesize,initial,step)
        gridax = gridfigure.add_subplot(rows,cols,i+1)
        gridax.imshow(matrixout, aspect='auto', interpolation='nearest', cmap='Greys')
        gridax.set_title(str(rules[i]))
        plt.axis('off')
        
    plt.show()
    
def plot_automata(timesteps,spacesize,initial,step,rule):
    """ Evaluates the automata for a specific rule

    Inputs:
    timesteps          - Duration of simulation
    spacesize          - Size of simulation
    initial            - Initial configuration of live cells
    step               - Step number
    rule               - Selected rule

    Output:
    Plots automata

    """
    
    # Plotting conditions
    windowsize = 10
    gridfigure = plt.figure(figsize = (windowsize,windowsize))
    plt.axis('off')

    # Evaluating with selected rule
    matrixout = run(rule,timesteps,spacesize,initial,step)
    plt.imshow(matrixout, aspect='auto', interpolation='nearest', cmap='Greys')
    plt.title(str(rule))
    plt.axis('off')
        
    plt.show()
    
# Initial conditions
timesteps = 10
spacesize = 10
initial = np.array([5]) # Placing live cells
step = 1
rule = (0,1,0,1,0)

### The following functions would then be run

In [ ]:
plot_automata(timesteps,spacesize,initial,step,rule)

In [ ]:
plot_automata_32(timesteps,spacesize,initial,step)

### 08/02/2022

Moving onto the 2-state automata in two dimensions, I begin with implementing similar constructions as with the one dimensional case. Only this time, for the purpose of understanding the outcome, I would need to animate the output. For this, I will use `matplotlib.animation`.

However, it is necessary to equally implement a mechanism of illustrating the evolution of the system in frames, so as to better compare and analyse the evolutions of the system. Furthermore, as opposed to initialising the space with a manual choise of 'live' cells, it would now be of convenience to develop a method of initialising the space with a random quantity of ones. I have chosen to implement this with the following:

    def initialise(probability):
   
        return np.random.choice((0,1), p=[1-probability,probability],size=(width,height))    
        
Where the input probability corresponds to the likelyhood of a cell initiating as one.

The update function now necessitates the values of all 8 surrounding cells to a given cell, and so this total is achieved via:

    def update2d(cell):

        space2d_updated = np.zeros((width,height))

        for j in range(1,height-1):
            for i in range(1,width-1):
                total_up = cell[i-1,j+1] + cell[i,j+1] + cell[i+1,j+1]
                total_middle = cell[i-1,j] + cell[i+1,j]          
                total_below = cell[i-1,j-1] + cell[i,j-1] + cell[i+1,j-1]
                total = total_up + total_middle + total_below
                space2d_updated[i,j] = rule2d(total,cell[i,j])

        return space2d_updated
        
This time we do not implement the value of the center cell, only the properties around it.

### 10/02/2022

Having not implemented rules yet, the game of life rules are implemented in the following way:

    def rule2d(total,cell):

        if cell == 1:
            if total == 2:     • 0
                cell = 1
            elif total == 3:   • 1
                cell = 1
            elif total <= 1:   • 2
                cell = 0
            elif total >= 4:   • 3
                cell = 0

        elif cell == 0:        • 4
            if total == 3:
                cell = 1

        return cell
        
The rules 0 to 4 correspond to the written equivalents:<br>

• 0: A live cell with two neighbours will stay alive at the next step.<br>
• 1: A live cell with three neighbours will stay alive at the next step.<br>
• 2: A live cell with fewer neighbours will die.<br>
• 3: A live cell with more neighbours will die.<br>
• 4: A dead cell which has exactly three neighbours will come to life at the next step.<br>

Implementing these two functions symbiotically, as done with the one dimensional version, produces successful results for both the animation and frame outputs. The testing 'glider' succesfully moves diagonally with time.

Once again boundary conditions are an important attribute as both the one dimensional and two dimensional 2-state automata misbehave when these are not correctly implemented. I have decided to choose non periodic boundaries and simply neglect the update function applying on the edge cells. For this reason the `i` and `j` dummy indices in the two for loops within the `update2d` function range from 1 to dimension+1 as opposed to 0 until the dimension.

It must also be noted that there is the need to create two matrices that contain information on the cells. The first contains the cells prior to the rules being applied, then the results are manifested on the second matrix. This enables the important element of updating every cell within the space <b>simultaneously</b>. And hence avoids overwriting on the cell that is in the midst of being analysed.

### 12/02/2022

I have now explored the 'Game of Life' simulation with different rules applied. Though the original rules prove themselves as the most interesting. I spent some time now experimenting with the best way to apply the mechanisms of this simulation to the forest fire. This time the automata will contain three states: 
<i>Green forest, Burning forest, Burnt-out forest</i>. The best way to encode the three states is important, as simply detecting totals will not differentiate a cell of value 2 from two neighbouring cells of value 1. I believe that the most intuitive approach seems to encode each cell with a value in multiples of 10 such that the digits will correspons to the state of the cell. However, discussions with the group suggested that there are other approaches. Though to facilitate the understanding of code and make a more generalised program, I will implement my own method. Also, I believe that the method of breaking down digits will facilitate extensions when needed.

# Part 2 - 2-State 2-Dimension GOL-Rules

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
import numpy as np
import random

%matplotlib inline

#Initialise grid
gridsize = 50
width = gridsize + 1
height = gridsize + 1
space2d = np.zeros((width,height))

def initialise(probability):
    """ Creates matrix of zeros where ones are placed randomly with a given probability (probability)
    
    Input:
    probability    - Probability of inserting a one
    
    Output: 
    Matrix of zeros containing ones with given probability
    """
    return np.random.choice((0,1), p=[1-probability,probability],size=(width,height))

#Initialising space with probability = 0.4 of ones occuring
space2d = initialise(0.4)

def rule2d(total,cell):
    """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 

    Inputs:
    total          - Sum of neighbouring cells
    cell           - Value of input cell

    Output:
    cell           - Value of the cell in following timestep

    """
    
    if cell == 1:
        if total == 2:
            cell = 1
        elif total == 3:
            cell = 1
        elif total <= 1:
            cell = 0
        elif total >= 4:
            cell = 0
               
    elif cell == 0:
        if total == 3:
            cell = 1

    return cell

def update2d(space2d):
    """ Evaluates the rules on each cell and its neighbours for an input matrix (space2d) 

    Input:
    space2d         - Input matrix that is interpreted cell by cell

    Output:
    space2d_updated - Output matrix for following timestep

    """
    
    space2d_updated = np.zeros((width,height))
    
    # Scanning every cell from input matrix and evaluating rules using neighbouring values
    for j in range(1,height-1):
        for i in range(1,width-1):
            total_up = space2d[i-1,j+1] + space2d[i,j+1] + space2d[i+1,j+1]
            total_middle = space2d[i-1,j] + space2d[i+1,j]          
            total_below = space2d[i-1,j-1] + space2d[i,j-1] + space2d[i+1,j-1]
            total = total_up + total_middle + total_below
            space2d_updated[i,j] = rule2d(total,space2d[i,j])
    
    return space2d_updated

# Initial conditions
windowsize = 15
rows = 10
cols = 10
time = 100

# Plotting conditions
gridfigure = plt.figure(figsize = (windowsize,windowsize))
plt.axis('off')

# Evaluating
matrixout = space2d
for i in range(0,time):
    matrixout_next = update2d(matrixout)
    gridax = gridfigure.add_subplot(rows,cols,i+1)
    gridax.imshow(matrixout_next, aspect='auto', interpolation='nearest', cmap='Greys')
    gridax.set_title(f't = {i}')
    plt.axis('off')
    matrixout = matrixout_next
    
plt.show()

### 16/02/2022

As a first means of approaching the forest fire simulation, I have decided to implement the states in the following form:<br>
<b>
<br>
Green forest → 100
<br>
Burning forest → 010
<br>
Burnt-out forest → 001
<br>
</b>

For example, a square whose diagonal cells correspond to green forest and whos above cell correspons to burning forest would read a total value of 413, that is: 4 green trees, 1 burning tree and the rest are absent.

In order read these digits, I decide to implement a scanning function that would take in this total value and output the digits independently, in turn allowing an update function to determine conditions based on the quantity of each state. This is written as:

    def total_scan(total):

        digit1 = (total%10)//1
        digit2 = (total%100)//10
        digit3 = (total%1000)//100

        return (digit1, digit2, digit3)
        
It is clear that any aditional states can trivially be implemented and so the code is now easily susceptible to further modifications.

Once again, we can create analagous rules and update functions as that of the 2-state 2-dimensional system, but now taking advantage of the state detecting function. Firstly, following the forest fire conditions, we define:


    def rule2d(total,cell):
     
        if cell == 100:             
            if int(total_scan(total)[1]) > 0:      • 0
                cell = 10
            elif prob(f) == 1:                     • 1
                cell = 10

        elif cell == 1:                            • 2
            if prob(p) == 1:
                cell = 100

        elif cell == 10:                           • 3
            cell = 1

        return cell
        
The rules 0 to 3 correspond to the written equivalents:<br>

• 0: If a cell is alight, then on the next step any neighbouring cell that was previously green forest will catch.<br>
• 1: Lightning may strike, allowing any green forest cell to catch fire with probability f per cell per timestep.<br>
• 2: Burnt-out forest sprouts green forest with a probability p per cell per time step.<br>
• 3: A cell that was alight will have burnt out.<br>

Furthermore, for the forest simulation we are told not to consider diagonal neighbouring cells, though I will explore this later. Hence the update function takes on the following structure:

    def update2d(space2d):

        space2d_updated = np.ones((w,h))

        for j in range(1,h-1):
            for i in range(1,w-1):
                total = space2d[i,j+1] + space2d[i-1,j] + space2d[i+1,j] + space2d[i,j-1]
                space2d_updated[i,j] = rule2d(total,space2d[i,j])

        return space2d_updated

### 17/02/2022

In order to facilitate the use of probabilities and reading of the code, I have now included the following simple function could also be recycled for any further explorations where I could experiment with the code.

    def prob(probability):

        return np.random.choice((0,1), p=[1-probability,probability])

Finally, the aesthetics of the output need to ideally correspond to the nature of the program. And so I have created a custom color map: `mpl.colors.ListedColormap(palette)`. This is inserted in the `cmap` attribute of `plt.imshow` where the variable 'palette' correspond to an array of selected colors:

    palette = ['black','red','green']

However, as the states are defined as increments of powers of 10, it is necessary to renormalise the color mapping to the states. A logarithmic reading is required and so I define: `mpl.colors.LogNorm(vmin=1, vmax=100)`. We have `vmax` set as the largest state and `vmin` as the smallest. As with the color map, this is inserted in the `norm` attribute of `plt.imshow`.

# Part 3 - 3-State 2-Dimension FF-Rules

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib as mpl
%matplotlib inline

#Initialise grid
gridsize = 10
width = gridsize + 1
height = gridsize + 1

def initialise(forest,burning):
    """ Creates matrix of ones where trees and burning trees are placed randomly with 
        given probabilities (forest), (burning)
    
    Inputs:
    forest     - Probability of a tree
    burning    - Probability of burning tree
    
    Output: 
    Matrix of ones containing 10, 100 with respective given probabilities
    """
    
    return np.random.choice((1,10,100), p=[1-forest-burning,burning,forest],size=(w,h))

# Random function
def prob(probability):
    """ Outputs a zero or a one with probability (probability)
    
    Input:
    probability    - Probability of outputting one
    
    Output: 
    0 or 1
    """
    
    return np.random.choice((0,1), p=[1-probability,probability])

#Initialising space with probabilities
space2d = initialise(0.8,0.001)

# Probability conditions
p = 0.05
f = 0.00025

# Choosing colors
palette = ['black','red','green']
cmap_custom = mpl.colors.ListedColormap(palette)
norm_custom = mpl.colors.LogNorm(vmin=1, vmax=100)

def total_scan(total):
    """ Outputs individual state count from an input total of these neighbours (total)
    
    Input:
    total       - value of all neighbouring cells
    
    Output: 
    Array containing each digit and hence each state
    """
    
    digit1 = (total%10)//1
    digit2 = (total%100)//10
    digit3 = (total%1000)//100
    
    return (digit1, digit2, digit3)

def rule2d(total,cell):
    """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 

    Inputs:
    total          - Sum of neighbouring cells
    cell           - Value of input cell

    Output:
    cell           - Value of the cell in following timestep

    """
    
    if cell == 100: # Green forest
        if int(total_scan(total)[1]) > 0:
            cell = 10
        elif prob(f) == 1:
            cell = 10
                
    elif cell == 1: # Burnt-out forest
        if prob(p) == 1:
            cell = 100
                
    elif cell == 10: # Burning forest
        cell = 1
                
    return cell

def update2d(space2d):
    """ Evaluates the rules on each cell and its neighbours for an input matrix (space2d) 

    Input:
    space2d            - Input matrix that is interpreted cell by cell

    Output:
    space2d_updated    - Output matrix for following timestep
    
    """
    
    space2d_updated = np.ones((width,height))
    
    # Scanning every cell from input matrix and evaluating rules using neighbouring values
    for j in range(1,height-1):
        for i in range(1,width-1):
            total = space2d[i,j+1] + space2d[i-1,j] + space2d[i+1,j] + space2d[i,j-1]
            space2d_updated[i,j] = rule2d(total,space2d[i,j])
    
    return space2d_updated

# Plotting conditions
windowsize = 15
rows = 10
cols = 10
time = 100

gridfigure = plt.figure(figsize = (windowsize,windowsize))
plt.axis('off')

# Evaluating
matrixout = space2d
for i in range(0,time):
    matrixout_next = update2d(matrixout)
    gridax = gridfigure.add_subplot(rows,cols,i+1)
    gridax.imshow(matrixout_next, aspect='auto', interpolation='nearest', cmap=cmap_custom, norm=norm_custom)
    gridax.set_title(f't = {i}')
    plt.axis('off')
    matrixout = matrixout_next
    
plt.show()

### 21/02/2022

Having completed all the main sections of the project, I will now explore some variations of the forest fire as the 3-state automata could do with some more realistic modifications. The first modification was to create two additional states. These were baby trees and water. Baby trees were mostly an aesthetic contribution as it seemed more logical for a baby tree to first spawn randomly and then grow into an adult tree on the following time step. These behave exactly like adult trees and can catch fire when neighbouring burning trees. Water simply blockades the cells, it cannot catch fire, nor can trees grow on it. All functions were modified to include these extensions, as well as the color map.

The 5-state automata now used the following mappings:<br>
<b>
<br>
Water → 10000
<br>
Adult forest → 01000
<br>
Baby forest → 00100
<br>
Burning forest → 00010
<br>
Burnt-out forest → 00001
<br>
</b>

Another suitable modification was that of additional probabilites. These included the increased chance of trees spawning around cells neighbouring water and the increased chance of trees spawning around cells with other neighbouring trees. The probabilities also increase depending on the number of their neighbouring tree/water cells.

The rule function, though significantly more complex, provides a more interesting set of rules:

    def rule2d(total,cell):

        prob_tree_plain = p + 3*p*int(total_scan(total)[2]) + 3*p*int(total_scan(total)[3])
        prob_tree_water = 50*(p + 3*p*int(total_scan(total)[2]) + 3*p*int(total_scan(total)[3]))

        if cell == 100:
            cell = 1000
            if int(total_scan(total)[1]) > 0:
                cell = 10

        elif cell == 1000:
            if int(total_scan(total)[1]) > 0:
                cell = 10
            elif prob(f) == 1:
                cell = 10

        elif cell == 1:
            if int(total_scan(total)[4]) > 0:
                if prob(prob_tree_water) == 1:
                    cell = 100
            elif prob(prob_tree_plain) == 1:
                cell = 100

        elif cell == 10:
            cell = 1

        return cell


# Part 3 - 5-State 2-Dimension FF-Rules Additional-1

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib as mpl
%matplotlib inline

#Initialise grid
gridsize = 30
width = gridsize + 1
height = gridsize + 1

def initialise(forest,burning,baby):
    """ Creates matrix of ones where adult trees, burning trees and baby trees are placed randomly with 
        given probabilities (forest), (burning), (baby)
    
    Inputs:
    forest     - Probability of an adult tree
    burning    - Probability of burning tree
    baby       - Probability of baby tree
    
    Output: 
    Matrix of ones containing 10, 100, 1000 with respective given probabilities
    """
    return np.random.choice((1,10,100,1000), p=[1-forest-burning-baby,burning,baby,forest],size=(width,height))

#Initialising space with probabilities
space2d = initialise(0.2,0.0,0.3)
space2d[8:12,:] = 10000 # Adding water, river

# Probability condition
p = 0.0001
f = 0.0005

# Random function
def prob(probability):
    """ Outputs a zero or a one with probability (probability)
    
    Input:
    probability    - Probability of outputting one
    
    Output: 
    0 or 1
    """
    
    return np.random.choice((0,1), p=[1-probability,probability])

# Choosing colors
palette = ['lightgreen','orangered','green','darkgreen','lightblue']
cmap_custom = mpl.colors.ListedColormap(palette)
norm_custom = mpl.colors.LogNorm(vmin=1, vmax=10000)

def total_scan(total):
    """ Outputs individual state count from an input total of these neighbours (total)
    
    Input:
    total       - value of all neighbouring cells
    
    Output: 
    Array containing each digit and hence each state
    """
    
    digit1 = (total%10)//1 # Burnt-out forest
    digit2 = (total%100)//10 # Burning forest
    digit3 = (total%1000)//100 # Baby tree
    digit4 = (total%10000)//1000 # Green forest
    digit5 = (total%100000)//10000 # Water
    
    return (digit1, digit2, digit3, digit4, digit5)

def rule2d(total,cell):
    """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 

    Inputs:
    total          - Sum of neighbouring cells
    cell           - Value of input cell

    Output:
    cell           - Value of the cell in following timestep

    """
    
    # Tree population increases more with more surrounding trees
    prob_tree_plain = p + 3*p*int(total_scan(total)[2]) + 3*p*int(total_scan(total)[3])
    prob_tree_water = 50*(p + 3*p*int(total_scan(total)[2]) + 3*p*int(total_scan(total)[3]))

    if cell == 100: # Baby tree
        cell = 1000
        if int(total_scan(total)[1]) > 0:
            cell = 10
            
    elif cell == 1000: # Green forest
        if int(total_scan(total)[1]) > 0:
            cell = 10
        elif prob(f) == 1:
            cell = 10
                
    elif cell == 1: # Burnt-out forest
        if int(total_scan(total)[4]) > 0:
            if prob(prob_tree_water) == 1:
                cell = 100
        elif prob(prob_tree_plain) == 1:
            cell = 100
                
    elif cell == 10: # Burning forest
        cell = 1
                
    return cell

# Specific color to cellin/out input 

def update2d(space2d):
    """ Evaluates the rules on each cell and its neighbours for an input matrix (space2d) 

    Input:
    space2d            - Input matrix that is interpreted cell by cell

    Output:
    space2d_updated    - Output matrix for following timestep
    
    """
    
    space2d_updated = np.ones((width,height))
                   
    for j in range(1,height-1):
        for i in range(1,width-1):
            total = space2d[i,j+1] + space2d[i-1,j] + space2d[i+1,j] + space2d[i,j-1]
            space2d_updated[i,j] = rule2d(total,space2d[i,j])
    
    return space2d_updated

# Plotting conditions
windowsize = 15
rows = 5
cols = 5
time = 25

gridfigure = plt.figure(figsize = (windowsize,windowsize))
plt.axis('off')

# Evaluating
matrixout = space2d
for i in range(0,time):
    matrixout_next = update2d(matrixout)
    gridax = gridfigure.add_subplot(rows,cols,i+1)
    gridax.imshow(matrixout_next, aspect='auto', interpolation='nearest', cmap=cmap_custom, norm=norm_custom)
    gridax.set_title(f't = {i}')
    plt.axis('off')
    matrixout = matrixout_next
    
plt.show()

### 28/02/2022

Another idea, is that of making the program more applicable on a practical level. I will attempt to use some libraries and `matplotlib` to interpret a selected image as the 2D space on which the forest fire will be evaluated. This could be thought as choosing an image of a region, say, a screenshot of a region in google maps. This will then make a best attempt to translate and break the image up into pixels and states corresponding to forest, water and plain. Then, one can use the simulation to 'predict' the outcome of a forest fire in this environment.

In order to begin this exploration, a suitable image translating library is required. I found the most comprehensible one to be `PIL`. A significant amount of trial and error produced the following image recognition and translation program: 

    sample = 30

    imageimport = Image.open(r"/Users/'name'/Desktop/10559c2e4208a957b7f8d7f1c82a5055.jpg")
    small = imageimport.resize((sample,sample),resample=Image.BILINEAR)
    result = small.resize((sample,sample))
    ar=np.array(result.convert('P', palette=Image.ADAPTIVE, colors=5))

    argrid=np.zeros((sample,sample))

    for j in range(0,sample):
            for i in range(0,sample):
                if ar[i,j] == 0:
                    argrid[i,j] = 10000
                if ar[i,j] == 1:
                    argrid[i,j] = 10000
                if ar[i,j] == 2:
                    argrid[i,j] = 100
                if ar[i,j] == 3:
                    argrid[i,j] = 100
                if ar[i,j] == 4:
                    argrid[i,j] = 1

    space2d = argrid.astype(int)
    
This will translate the imported image into a <b>sample</b> $\times$ <b>sample</b> sized grid, pixelated suitably to adapt to this. It will then translate at a best attempt the information into a matrix, readable by the previous program. The results, though not specifically detecting colors, through a multitude of readjustments, can produce some realtively accurate translations of images. 

However, in order to simulate the randomness of tree spawning in the non-watered areas, I add the variable 'texturing'. This will then apply a probability that the trees will spawn on the terrestial areas and help simulate the randomness. 

# Part 3 - 5-State 2-Dimension FF-Rules Additional-2

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib as mpl
from PIL import Image
%matplotlib inline

# Choosing colors
palette = ['lightgreen','orangered','green','darkgreen','lightblue']
cmap_custom = mpl.colors.ListedColormap(palette)
norm_custom = mpl.colors.LogNorm(vmin=1, vmax=10000)

# Importing image and conversion

sample = 30
texturing = 0.7

imageimport = Image.open(r"/Users/'name'/Desktop/10559c2e4208a957b7f8d7f1c82a5055.jpg")
small = imageimport.resize((sample,sample),resample=Image.BILINEAR)
result = small.resize((sample,sample))
ar=np.array(result.convert('P', palette=Image.ADAPTIVE, colors=5))

argrid=np.zeros((sample,sample))

for j in range(0,sample):
        for i in range(0,sample):
            if ar[i,j] == 0:
                if prob(texturing) == 0:
                    argrid[i,j] = 1000
                elif prob(texturing) == 0:
                    argrid[i,j] = 100
                else: argrid[i,j] = 1
            if ar[i,j] == 1:
                if prob(texturing) == 0:
                    argrid[i,j] = 1000
                elif prob(texturing) == 0:
                    argrid[i,j] = 100
                else: argrid[i,j] = 1
            if ar[i,j] == 2:
                if prob(texturing) == 0:
                    argrid[i,j] = 1000
                elif prob(texturing) == 0:
                    argrid[i,j] = 100
                else: argrid[i,j] = 1
            if ar[i,j] == 3:
                argrid[i,j] = 10000
            if ar[i,j] == 4:
                if prob(texturing) == 0:
                    argrid[i,j] = 1000
                elif prob(texturing) == 0:
                    argrid[i,j] = 100
                else: argrid[i,j] = 1
                
space2d = argrid.astype(int)

#Initialise grid
gridsize = sample-1
width = gridsize + 1
height = gridsize + 1

# Probability condition
p = 0.0001
f = 0.0005

# Random function
def prob(probability):
    """ Outputs a zero or a one with probability (probability)
    
    Input:
    probability    - Probability of outputting one
    
    Output: 
    0 or 1
    """
    
    return np.random.choice((0,1), p=[1-probability,probability])

def total_scan(total):
    """ Outputs individual state count from an input total of these neighbours (total)
    
    Input:
    total       - value of all neighbouring cells
    
    Output: 
    Array containing each digit and hence each state
    """
    
    digit1 = (total%10)//1 # Burnt-out forest
    digit2 = (total%100)//10 # Burning forest
    digit3 = (total%1000)//100 # Baby tree
    digit4 = (total%10000)//1000 # Green forest
    digit5 = (total%100000)//10000 # Water
    
    return (digit1, digit2, digit3, digit4, digit5)

def rule2d(total,cell):
    """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 

    Inputs:
    total          - Sum of neighbouring cells
    cell           - Value of input cell

    Output:
    cell           - Value of the cell in following timestep

    """
    
    # Tree population increases more with more surrounding trees
    prob_tree_plain = p + 3*p*int(total_scan(total)[2]) + 3*p*int(total_scan(total)[3])
    prob_tree_water = 50*(p + 3*p*int(total_scan(total)[2]) + 3*p*int(total_scan(total)[3]))

    if cell == 100: # Baby tree
        cell = 1000
        if int(total_scan(total)[1]) > 0:
            cell = 10
            
    elif cell == 1000: # Green forest
        if int(total_scan(total)[1]) > 0:
            cell = 10
        elif prob(f) == 1:
            cell = 10
                
    elif cell == 1: # Burnt-out forest
        if int(total_scan(total)[4]) > 0:
            if prob(prob_tree_water) == 1:
                cell = 100
        elif prob(prob_tree_plain) == 1:
            cell = 100
                
    elif cell == 10: # Burning forest
        cell = 1
                
    return cell

# Specific color to cellin/out input 

def update2d(space2d):
    """ Evaluates the rules on each cell and its neighbours for an input matrix (space2d) 

    Input:
    space2d            - Input matrix that is interpreted cell by cell

    Output:
    space2d_updated    - Output matrix for following timestep
    
    """
    
    space2d_updated = np.ones((width,height))
                   
    for j in range(1,height-1):
        for i in range(1,width-1):
            total = space2d[i,j+1] + space2d[i-1,j] + space2d[i+1,j] + space2d[i,j-1]
            space2d_updated[i,j] = rule2d(total,space2d[i,j])
    
    return space2d_updated

# Plotting conditions
windowsize = 15
rows = 5
cols = 5
time = 25

gridfigure = plt.figure(figsize = (windowsize,windowsize))
plt.axis('off')

# Evaluating
matrixout = space2d
for i in range(0,time):
    matrixout_next = update2d(matrixout)
    gridax = gridfigure.add_subplot(rows,cols,i+1)
    gridax.imshow(matrixout_next, aspect='auto', interpolation='nearest', cmap=cmap_custom, norm=norm_custom)
    gridax.set_title(f't = {i}')
    plt.axis('off')
    matrixout = matrixout_next
    
plt.show()

### 25/02/2022

A new idea came about, which is to explore how the 2-state 2-dimensional automata would behave under various conditions, such that one could find the most suitable initial setup that would produce the longest lasting simulation. This would require the plotting of the cell count as a graph, to better visualise, as well as taking into account the average cell count over a given duration. 

The strategy is to create a function that will intake the grid size, the live spell probability for the initial state and finally the duration of the simulation.

For given gridsizes and durations, one can cycle through the various probabilities and calculate the average cell count. Plotting this date may help deduce the ideal probabilities for the highest cell count.

In [ ]:
# Necessary imports
import matplotlib.pyplot as plt
import numpy as np
import random

%matplotlib inline

def initialise(probability):
    """ Creates matrix of zeros where ones are placed randomly with a given probability (probability)
    
    Input:
    probability    - Probability of inserting a one
    
    Output: 
    Matrix of zeros containing ones with given probability
    """
    return np.random.choice((0,1), p=[1-probability,probability],size=(width,height))

def rule2d(total,cell):
    """ Evaluates the rules on a cell (cell) using the sum of its neighbour values (total) 

    Inputs:
    total          - Sum of neighbouring cells
    cell           - Value of input cell

    Output:
    cell           - Value of the cell in following timestep

    """
    
    if cell == 1:
        if total == 2:
            cell = 1
        elif total == 3:
            cell = 1
        elif total <= 1:
            cell = 0
        elif total >= 4:
            cell = 0
               
    elif cell == 0:
        if total == 3:
            cell = 1

    return cell

def update2d(space2d):
    """ Evaluates the rules on each cell and its neighbours for an input matrix (space2d) 

    Input:
    space2d         - Input matrix that is interpreted cell by cell

    Output:
    space2d_updated - Output matrix for following timestep

    """
    
    space2d_updated = np.zeros((width,height))
    
    # Scanning every cell from input matrix and evaluating rules using neighbouring values
    for j in range(1,height-1):
        for i in range(1,width-1):
            total_up = space2d[i-1,j+1] + space2d[i,j+1] + space2d[i+1,j+1]
            total_middle = space2d[i-1,j] + space2d[i+1,j]          
            total_below = space2d[i-1,j-1] + space2d[i,j-1] + space2d[i+1,j-1]
            total = total_up + total_middle + total_below
            space2d_updated[i,j] = rule2d(total,space2d[i,j])
    
    return space2d_updated

def simulate(gridsize, p, time):

    #Initialise grid
    width = gridsize + 1
    height = gridsize + 1
    space2d = np.zeros((width,height))

    # Initialising space with probability = 0.5 of live cells
    space2d = initialise(p)

    # Plotting conditions
    windowsize = 5
    time = 100

    cellcount = np.zeros(time)

    # Evaluating
    matrixout = space2d
    cellcount[0] = np.sum(matrixout)

    for i in range(0,time):
        matrixout_next = update2d(matrixout)
        cellcount[i] = np.sum(matrixout)
        matrixout = matrixout_next
        
    finalcell = int(cellcount[-1])
    averagecell = np.sum(cellcount)/len(cellcount)

    title = (f'Final = {finalcell} | Average = {averagecell}')
    
    return cellcount, averagecell

# Initial conditions
probabilities = 20
repeats = 5
probarray = np.linspace(0,1,probabilities+1)
size = 20
steps = 100

averages = np.zeros((probabilities+1,repeats))

for r in range(0,repeats):
    for i in range(0,probabilities):
        probvalue = i*(1/probabilities)
        averages[i,r] = simulate(size,probvalue,steps)[1]
    
combinedaverage = np.sum(averages,axis=1)/repeats
maxloc = np.where(combinedaverage == np.max(combinedaverage))

plt.figure(figsize = (10,5))
plt.plot(probarray, combinedaverage, color = 'grey')
plt.plot(probarray,averages,color='lightgrey',linestyle='dashed')
plt.axvline(float(probarray[maxloc]), color='red', linestyle='dashed')
plt.scatter(float(probarray[maxloc]), np.max(combinedaverage), color='red')
plt.xlabel(f'Live cell initial probability | Max p = {round(float(probarray[maxloc]),2)}')
plt.ylabel(f'Average cell count | Max count = {round(np.max(combinedaverage),2)}')
plt.title(f'Average cell count by initial probability after {steps} steps in a {size}x{size} grid.')
plt.show()